## 16. 코루틴
### 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

예제 16-1은 코루틴의 동작을 보여준다.

In [1]:
""" [예제 16-1] 가장 간단한 코루틴 사용 """
from inspect import getgeneratorstate

def simple_coroutine(): # 코루틴은 자신의 본체 안에 yield문을 가진 일종의 제너레이터 함수로 정의된다.
    print('-> coroutine started')
    x = yield # yield를 표현식에 사용한다. 단지 호출자에 데이터를 받도록 설계하면 yield는 값을 생성하지 않는다.
    print('-> coroutine received:', x)
    
my_coro = simple_coroutine() # 일반적인 제너레이터와 마찬가지로 함수를 호출해서 제너레이터 객체를 가져온다.
print(repr(my_coro))

<generator object simple_coroutine at 0x000002E599D62990>


In [2]:
print(getgeneratorstate(my_coro))
next(my_coro) # next()를 호출해서 제너레이터를 yield문 까지 실행함으로써 데이터를 전송할 수 있는 상태를 만든다.
print(getgeneratorstate(my_coro))

GEN_CREATED
-> coroutine started
GEN_SUSPENDED


In [3]:
print(getgeneratorstate(my_coro))
my_coro.send(42) # 제너레이터의 send() 메서드를 호출해서 코루틴 본체 안의 yield 문의 값을 42로 만든다.
                 # 이제 코루틴이 실행을 재개해서 다음 yield문이 나오거나 종료될 때까지 실행한다.
                 # 제어흐름이 코루틴 본체의 끝에 도달하므로, 일반적인 제너레이터와 마찬가지로 StopIteration 예외를 발생시킨다.

GEN_SUSPENDED
-> coroutine received: 42


StopIteration: 

In [4]:
print(getgeneratorstate(my_coro))

GEN_CLOSED


코루틴은 네 가지 상태를 가진다. inspect.getgeneratorstate( ) 함수를 이용해서 현재 상태를 알 수 있다.
+ GEN_CREATE    : 실행을 시작하기 위해 대기하고 있는 상태
+ GEN_RUNNING   : 현재 인터프리터가 실행하고 있는 상태
+ GEN_SUSPERNED : yield문에서 대기하고 있는 상태
+ GEN_CLOSED    : 실행이 완료된 상태

코루틴 객체를 생성하고 난 직후(GEN_CREATE 상태)에 바로 None이 아닌 값을 전달하려고 하면 다음과 같은 오류가 발생한다. 처음 next(my_coro)를 호출할 때, 코루틴을 기동<sub>priming</sub> 한다고 표현한다. 즉, 코루틴이 호출자로부터 값을 받을 수 있도록 처음 yield문까지 실행을 진행하는 것이다.

In [5]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

In [6]:
""" [예제 16-2] 두 번 생성하는 코루틴 """

def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [7]:
next(my_coro2)

-> Started: a = 14


14

In [8]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [9]:
my_coro2.send(42)

-> Received: b = 42


56

In [10]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [11]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

[그림 16-1] simple_coro2 코루틴을 실행하는 3 단계. 각 단계는 yield 표현식에서 끝나며, 다음 단계는 yield 표현식의 값을 변수에 할당하는 동일 행에서 시작한다. 
<img src="Figure16-1.png">

### 16.3 이동 평균을 계산하는 코루틴
예제 7-14에서 클로저를 생성해서 total과 count 변수를 보전하는 고급함수와 비교해보자.

In [12]:
""" [예제 7-14] nonlocal 키워드로 오류 해결 """
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager() 
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [13]:
""" [예제 16-3, 4] 이동 평균 코루틴과 doctest """
def averager():
    count = 0
    total = 0
    average = None
    
    while True: # 무한 푸르이므로 이 코루틴은 호출자가 값을 보내주는 한 계속해서 값을 받고 결과를 생성한다. 이 코루틴은 호출자가 close() 메서드를 호출하거나,
                # 이 객체에 대한 참조가 모두 사라져서 가비지 컬렉트되어야 종료된다. 
        term = yield average # 이 yield 문은 코루틴을 중단하고 지금까지의 평균을 생성하기 위해 사용된다. 
        total += term
        count += 1
        average = total/count

coro_avg = averager()
next(coro_avg)

In [14]:
coro_avg.send(18)

18.0

In [15]:
coro_avg.send(30)

24.0

In [16]:
coro_avg.send(5)

17.666666666666668

### 16.4 코루틴을 기동하기 위한 데커레이터
코루틴을 편리하게 사용할 수 있도록 기동하는 데커레이터가 종종 사용된다. @coroutine가 대표적이다.

In [17]:
""" [예제 16-5] 코루틴을 기동하기 위한 데커레이터. (yield from과 함께 사용 불가능) """
from functools import wraps

def coroutine(func):
    """ 데커레이터: 'func'를 기동해서 첫 번째 'yield'까지 진행한다. """
    @wraps(func)                    # 데커레이트된 제너레이터 함수는 primer() 함수로 치환되며, 실행하면 기동된 제너레이터를 반환한다.
    def primer(*args, **kwargs):    # 데커레이트된 함수를 호출해서 제너레이터 객체를 가져온다.
        gen = func(*args, **kwargs) # 제너레이터를 기동한다.
        next(gen)                   # 제너레이터를 반환한다.
        return gen
    return primer

In [18]:
@coroutine
def averager():
    count = 0
    total = 0
    average = None
    
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count

In [19]:
coro_avg = averager()

from inspect import getgeneratorstate
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [20]:
coro_avg.send(18)

18.0

In [21]:
coro_avg.send(30)

24.0

In [22]:
coro_avg.send(5)

17.666666666666668

### 16.5 코루틴 종료와 예외 처리
코루틴 안에서 발생한 예외를 처리하지 않으면, next( )나 send( )로 코루틴을 호출한 호출자에 예외가 전파된다.

In [23]:
""" [예제 16-7] 처리하지 않은 예외에 의한 코루틴 종료 """
coro_avg = averager()
coro_avg.send(40)
coro_avg.send(50)

45.0

In [24]:
coro_avg.send('spam') # 코루틴 내에서 예외 발생

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [25]:
coro_avg.send(60) # 코루틴 안에서 예외를 처리하지 않으면 루프가 종료되고 다시 활성화되지 않는다.

StopIteration: 

[예제 16-7]을 보면 종료하라고 코루틴에 알려주는 구분 표시를 전송해서 코루틴을 종료할 수 있음을 알 수 있다. None이나 Eliipsis와 같은 내장된 싱글턴 상수는 구분표시로 사용하기 좋다. (필자는 my_coro.send(StopIteration) 형태로 사용하기도 한다.)

파이선 2.5 이후 제너레이터 객체는 호출자가 코루티네 명시적으로 예외를 전달할 수 있도록 throw( )와 close( ) 메서드를 제공한다.

```
generator.throw(exc_type[, exc_value[, traceback]])
제너레이터가 중단한 곳의 yield 표현식에 예외를 전달한다. 제너레이터가 예외를 처리하면 제어흐름이 다음 yield 문까지 진행하고 생성된 값은 generator.throw() 호출 값이 된다. 제너레이터가 예외를 처리하지 않으면 호출자까지 예외가 전파된다.
```
```
generator.close( )
제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 발생시키게 만든다. 제너레이터가 예외를 처리하지 않거나 StopIteration 예외(일반적으로 제너레이터가 실행을 완료할 때 발생한다.)를 발생시키면 아무런 에러도 호출자에 전달되지 않는다. GeneratorExit 예외를 받으면 제너레이터는 아무런 값도 생성하지 않아야 한다. 아니면 RuntimeError 예외가 발생한다. 제너레이터에서 발생하는 다른 예외는 모두 호출자에 전달된다.
```

In [26]:
""" [예제 16-8] 코루틴의 예외 처리방법을 설명하기 위한 제너레이터 """

class DemoException(Exception):
    """ 설명에 사용할 예외 유형 """
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException: # 예외를 따로 처리
            print('*** DemoException handled. Continuing... ')
        else: # 예외가 발생되지 않으면 받은 값을 출력
            print('-> coroutine received: {!r}'.format(x))
#         raise RuntimeError('This line should never run.') # 무한 루프는 처리되지 않은 예외에 의해서만 중단될 수 있으며,
#                                                           # 예외를 처리하지 않으면 코루틴의 실행이 바로 중단되므로 마지막 줄은 실행될 수 없다.
#                                                           # 책과는 다르게 인터프리터가 마지막 줄은 실행될 수 없음을 알림(Upgrade 된듯)

In [27]:
""" [예제 16-9] 예외를 발생시키지 않은 demo_exc_handling의 활성화 및 종료 """

exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [28]:
exc_coro.send(11)

-> coroutine received: 11


In [29]:
exc_coro.send(22)

-> coroutine received: 22


In [30]:
exc_coro.close()

In [31]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [32]:
""" [예제 16-10] DemoException을 demo_exc_handling 안에 던져도 종료되지 않음 """

exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [33]:
exc_coro.send(11)

-> coroutine received: 11


In [34]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing... 


In [35]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [36]:
""" [예제 16-11] 자신에게 던져진 예외를 처리할 수 없으면 코루틴이 종료됨 """
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(ZeroDivisionError)

-> coroutine started
-> coroutine received: 11


ZeroDivisionError: 

In [37]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

코루틴이 어떻게 종료되든 정리 코드를 실행해야 하는 경우에는 try/finally 블록 안에 코루틴의 해당 코드를 넣어야 한다.

In [38]:
""" [예제 16-12] 코루틴 종료 시 정리작업을 실행하기 위핸 try/finally문 추가 """

class DemoException(Exception):
    """ 설명에 사용할 예외 유형 """
    
def demo_finally():
    print('-> coroutine started')
    
    try:
        while True:
            try:
                x = yield
            except DemoException: # 예외를 따로 처리
                print('*** DemoException handled. Continuing... ')
            else: # 예외가 발생되지 않으면 받은 값을 출력
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [39]:
exc_coro = demo_finally()
next(exc_coro)
exc_coro.throw(ZeroDivisionError)

-> coroutine started
-> coroutine ending


ZeroDivisionError: 

### 16.6 코루틴에서 값 반환하기
예제 16-13은 항목 수(count)와 평균(average)을 담은 namedtuple을 반환한다.

In [40]:
""" [예제 16-13] averager( ) 코루틴 코드 """

from collections import namedtuple
Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다. 그렇기 때문에 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average) # 파이선 3.3 이전 버전에서는 제너레이터가 값을 반환하므로 에러가 발생한다.  

In [41]:
""" [예제 16-14] averager( )의 동작을 보여주는 doctest """

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None) # 일반적인 제너레이터 객체와 같이 StopIteration 예외가 발생하며 value 속성에 반환값이 들어 있다.

StopIteration: Result(count=3, average=15.5)

In [42]:
""" [예제 16-15] StopIteration 예외 처리를 통한 반환값 가져오기 """

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

try: # try문을 통한 우회적인 처리 방법
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
    
print(result)

Result(count=3, average=15.5)


### 16.7 yield from 사용하기
yield from은 완전히 새로운 언어 구성체라는 점을 잊지말자. (다른 언어에서는 이와 비슷한 구성체를 await라고 표현한다.) 

yield from x 표현식이 x 객체에 대해 첫 번째로 하는 일은 iter(x)를 호출해서 x의 반복자를 가져오는 것이다. 이는 모든 반복형이 x에 사용될 수 있다는 의미이다.

In [43]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
        
list(gen())

['A', 'B', 1, 2]

In [44]:
def gen():
    yield from 'AB'
    yield from range(1, 3)
        
list(gen())

['A', 'B', 1, 2]

그러나 내포된 for 루프를 대체하는 것보다 더 주요한 특징은 가장 바깥쪽 호출자와 가장 안쪽 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다. 따라서 이 둘이 값을 직접 주고 받으며 중간에 있는 코루틴이 판에 박힌 듯한 예외처리 코드를 구현할 필요 없이 예외를 직접 던질 수 있다.이를 코루틴 위임<sup>coroutine delegation</sup>이라 한다.

아래는 중요 용어에 대한 설명이다.
+ 대표 제너레이터(delegating generator)
    + yield from <반복형> 표현식을 담고 있는 제너레이터 함수
+ 하위 제너레이터(subgenerator)
    + yield from 표현식 중 <반복형>에서 가져오는 제너레이터
+ 호출자(caller)
    + 대표 제너레이터를 호출하는 코드. 문맥에 따라서 필자는 대표 제너레이터와 구분하기 위해 '호출자' 대신 '클라이언트'라는 용어를 사용하기도 한다. 하위 제너레이터 입장에서 보면 대표 제너레이터도 호출자이기 때문이다.
    
[그림 16-2] 대표 제너레이터가 yield from에서 중단하고 있는 동안, 호출자는 하위 제너레이터에 데이터를 직접 전송하고 하위 제너레이터는 다시 데이터를 생성해서 호출자에게 전달한다. 하위 제너레이터가 실행을 완료하고 인터프리터가 반환된 값을 첨부한 StopIteration을 발생시키면 대표 제너레이터가 실행을 재개한다.

<img src="Figure16-2.png">

In [45]:
""" [예제 16-17] yield from을 이용해서 averager()를 구동하고 보고서 생성하기 """

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 하위 제너레이터
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield # term에 데이터를 받을 수 있음
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# 대표 제너레이터
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# 호출자(클라이언트)
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key) # result로 하위 제너레이터에서 계산한 값을 받고, key는 대표제너레이터에 정보를 전달한다.
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # 이 부분이 중요하다!
        
    print(results) # for debug
    report(results)
    
# 실행결과 보고서
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'
              .format(result.count, group, result.average, unit))

data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m' : [ 1.6, 1.51,  1.4,  1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg' : [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m'  : [1.38,  1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],    
}

if __name__ == '__main__':
    main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
